#  PERM data analysis using R

---

# Read and Clean the data

In [1]:
getwd()

[1] "/Users/Leo/GitHub/Data-Analysis-Example"

In [2]:
file_list <- list.files(file.path(getwd(), 'data/all_perm'), pattern = "*.csv", full.names = TRUE)
length(file_list)

[1] 177

In [3]:
read_csv_filename <- function(filename){
    ret <- read.csv(filename, header=FALSE, sep=",", na.strings="" )
    ret$Country <- gsub("[+]", " ", strsplit(basename(file.path(filename)), "_", fixed = TRUE)[[1]][1])
    ret
}

original_data <- data.frame(do.call(rbind,lapply(file_list,read_csv_filename)))
colnames(original_data) <- c('id', 'Date', 'Employer', 'City_State', 'Status', 'Job_Title', 'Wage_Offer','Country')
str(original_data)

'data.frame':	740236 obs. of  8 variables:
 $ id        : Factor w/ 734586 levels "A-05312-50869",..: 52 53 51 44 50 49 48 42 45 47 ...
 $ Date      : Factor w/ 2957 levels "2008-02-08","2008-02-13",..: 53 52 51 50 49 48 47 46 45 44 ...
 $ Employer  : Factor w/ 154201 levels "1700 Pharmacy Inc",..: 30 40 9 25 19 19 28 8 1 38 ...
 $ City_State: Factor w/ 16643 levels "Aberdeen, NJ",..: 42 13 5 34 18 17 22 2 27 13 ...
 $ Status    : Factor w/ 5 levels "Certified","Certified-expired",..: 1 3 3 3 3 3 3 3 1 1 ...
 $ Job_Title : Factor w/ 61050 levels "Accountants and Auditors, Level IV",..: 29 26 27 33 5 9 1 28 34 24 ...
 $ Wage_Offer: Factor w/ 101446 levels "100/","1000/BI",..: 13 26 18 9 44 25 6 50 17 44 ...
 $ Country   : chr  "Afghanistan" "Afghanistan" "Afghanistan" "Afghanistan" ...


In [4]:
head(original_data, 3)

id,Date,Employer,City_State,Status,Job_Title,Wage_Offer,Country
A-16309-68487,2017-06-09,Saad S. Khan,"WILMINGTON, DE",Certified,"LIVE-IN-DOMESTIC, Level I","17,701.00/Year",Afghanistan
A-17003-86706,2017-03-20,Town & Country Properties Inc. & Subsidiaries,"Falls Church, VA",Certified-Expired,"Insurance Clerk, Level II","31,242.00/Year",Afghanistan
A-16293-63321,2017-01-31,"Cogent Medical Care, Pc","Binghamton, NY",Certified-Expired,"Internists, General, Level I","226,800.00/Year",Afghanistan


In [5]:
# clean the data, column by column

# Drop id
perm_data= subset(original_data, select = -c(id) )

# Parse dates
perm_data$Date <- as.Date(perm_data$Date)
# Sort by date
# perm_data <- perm_data[order(perm_data$Date, decreasing = TRUE, index.return = TRUE),]

In [6]:
# Employer name
library(plyr)

original_counts <- count(perm_data, 'Employer')
original_counts <- original_counts[order(original_counts$freq,decreasing = TRUE),]
head(original_counts, 10)

cat('--------------------------------\n')
cat('Employer names need to clean up.\n')
cat('--------------------------------\n')

head(original_counts[grepl("Apple ", original_counts[["Employer"]]) | 
                     grepl("Apple,", original_counts[["Employer"]]),], 5)

,Employer,freq
912,Cognizant Technology Solutions Us Corporation,20506
24,Microsoft Corporation,18298
450,Intel Corporation,8893
175,Google Inc.,5086
382,"Cisco Systems, Inc.",4742
580,Amazon Corporate Llc,4085
412,"Oracle America, Inc.",3207
1271,"Google, Inc.",2602
2911,Apple Inc.,2447
3459,Infosys Ltd.,2390


--------------------------------
Employer names need to clean up.
--------------------------------


,Employer,freq
2911,Apple Inc.,2447
614,"Apple, Inc.",1418
13321,"Apple Computer, Inc.",20
68,Apple Educational Services,10
117229,Red Apple Child Development Center,6


In [7]:
# first will be remove dba (dba, D/b/a , d/b/a)
# "\\b[[:punct:]]*d[[:punct:]]*b[[:punct:]]*a[[:punct:]]*\\b"

# lower case
perm_data$Employer <- sapply(perm_data$Employer, tolower)
# words before "dba"
perm_data$Employer <- sapply(perm_data$Employer, function(x) 
                        strsplit(x, "\\b[[:punct:]]*d[[:punct:]]b[[:punct:]]a[[:punct:]]*\\b")[[1]][1])
# remove special characters
perm_data$Employer <- gsub("[[:punct:]]", "", perm_data$Employer)

# Capitalizes the first letter of each word
perm_data$Employer <- sapply(perm_data$Employer, function(x) {
                                  s <- strsplit(x, " ")[[1]]
                                  paste(toupper(substring(s, 1,1)), substring(s, 2), sep="", collapse=" ")}
                              )     

# need to think about how to handel typo like: 'buisness' -> 'business'
# Or company rename: 'apple computer' -> 'apple'

In [8]:
new_counts <- count(perm_data, 'Employer')
new_counts <- new_counts[order(new_counts$freq, decreasing = TRUE),]
head(new_counts, 10)

,Employer,freq
28378,Cognizant Technology Solutions Us Corporation,20506
85153,Microsoft Corporation,18298
63080,Intel Corporation,8893
52803,Google Inc,7688
7095,Amazon Corporate Llc,5500
26815,Cisco Systems Inc,4742
9889,Apple Inc,3865
96027,Oracle America Inc,3207
35000,Deloitte Consulting Llp,2624
56501,Hcl America Inc,2418


In [12]:
head(perm_data, 3)

Date,Employer,City_State,Status,Job_Title,Wage_Offer,Country
2017-06-09,Saad S Khan,"WILMINGTON, DE",Certified,"LIVE-IN-DOMESTIC, Level I","17,701.00/Year",Afghanistan
2017-03-20,Town Country Properties Inc Subsidiaries,"Falls Church, VA",Certified-Expired,"Insurance Clerk, Level II","31,242.00/Year",Afghanistan
2017-01-31,Cogent Medical Care Pc,"Binghamton, NY",Certified-Expired,"Internists, General, Level I","226,800.00/Year",Afghanistan


In [54]:
# city and state, add area
test <- perm_data[grepl(",{2}", perm_data[["City_State"]]),]
str(test)

'data.frame':	312 obs. of  7 variables:
 $ Date      : Date, format: "2011-02-04" "2010-05-28" ...
 $ Employer  : chr  "Spellmans Marine Inc" "Capital University" "Mark Seth Adler Cpa Inc Dba Adler  Associates" "The University Of Chicago" ...
 $ City_State: Factor w/ 16643 levels "Aberdeen, NJ",..: 608 473 1135 1626 1636 1698 1698 1915 2146 2192 ...
 $ Status    : Factor w/ 5 levels "Certified","Certified-expired",..: 3 1 3 3 2 3 4 2 3 3 ...
 $ Job_Title : Factor w/ 61050 levels "Accountants and Auditors, Level IV",..: 1476 234 56 3072 3146 3244 3244 1168 305 4068 ...
 $ Wage_Offer: Factor w/ 101446 levels "100/","1000/BI",..: 1094 1880 2688 4127 4985 5858 5858 298 388 6805 ...
 $ Country   : chr  "Argentina" "Argentina" "Armenia" "Azerbaijan" ...


In [55]:

# 213 missing City_State
# 26 only have states name (without ",") 
# 739685 have one ","
# 312 have two "," and 4 cases start with "," : , Villa Platte, LA


test

,Date,Employer,City_State,Status,Job_Title,Wage_Offer,Country
2316,2011-02-04,Spellmans Marine Inc,"Hampton Bays,, NY",Certified-Expired,"Motorboat Mechanic, Level I",17/hr,Argentina
2587,2010-05-28,Capital University,"Columbus,, OH",Certified,"Mathematical Science Teachers, Postsecondary, Level I",56010/yr,Argentina
3976,2010-02-18,Mark Seth Adler Cpa Inc Dba Adler Associates,"San Diego,, CA",Certified-Expired,"Accountants, Level II",53269/yr,Armenia
7869,2013-09-18,The University Of Chicago,"Chicago,, IL",Certified-Expired,"Social Work Teachers, Postsecondary, Level I",73000/,Azerbaijan
8156,2007-10-09,Hcbeck Ltd,"Atlanta,, GA",Certified-expired,"Project Leader, LEVEL II",60500/YR,Bahamas
9765,2011-02-01,7eleven Franchise,"Alexandria,, VA",Certified-Expired,"Cashiers, Level I",7.78/hr,Bangladesh
9911,2010-07-02,7eleven Franchise,"Alexandria,, VA",Denied,"Cashiers, Level I",7.78/hr,Bangladesh
10259,2009-03-06,Lsu Health Sciences Center,"New Orleans,, LA",Certified-expired,"Health Specialties Teachers, Postsecondary, Level I",65000/yr,Bangladesh
11218,2014-02-03,Epam Systems Inc,"San Francisco,, CA",Certified-Expired,"Software Developers, Applications, Level II",92789/Year,Belarus
12019,2010-02-26,Usfc Inc Dba Us Floor Covering,"Laguna Hills,, CA",Certified-Expired,"Tile And Marble Setters, Level IV",26.47/hr,Belgium


In [10]:
# city and state, add area

# edit delimiter
# count how many without or have more than one ","

# split by ","

# perm_data$City  = sapply(perm_data$City_State, function(x) strsplit(x, ", ")[[1]][1])
# perm_data$State = sapply(perm_data$City_State, function(x) strsplit(x, ", ")[[1]][2])


# think about typo like missing state name, simply wrong text

# using city names list to add area
# The New York Metro area and California Bay Area cities list is from wikipedia

# ny_cities = read_csv('data/NY_cities.csv', names=['city'])
# bay_cities = read_csv('data/Bay_Area_cities.csv', names=['city'])

# perm.loc[(perm.State.isin(['NY', 'NJ', 'CT'])) & 
#          (perm.City.str.upper().isin(ny_cities.city)), 'Area']  = 'New York Metro'
# perm.loc[(perm.State.isin(['CA'])) & 
#          (perm.City.str.upper().isin(bay_cities.city)), 'Area'] = 'California Bay Area'


        Certified Certified-expired Certified-Expired            Denied 
           197498             38943            404088             65862 
        Withdrawn 
            33845 

In [41]:
# remove unnecessary characters
perm_data$Employer <- as.character(perm_data$Employer)

perm_data$test2 <- sapply(perm_data$Employer, function(x) unlist(strsplit(x, ","))[1])
perm_data$test2 <- gsub("[[:punct:]]", "", perm_data$test2)
head(perm_data)
                          
                          
# replace_list = [',', '!', '.', ' inc', ' llc', ' llp', ' ltd', ' lp', 
#                 ' corporation', ' corporate', ' corp', ' technologies'] 
# for char in replace_list: 
#     perm.Employer = perm.Employer.str.lower().str.replace(char, '')  

# edit employer name to operating name of a company, as opposed to the legal name of the company (dba)

,Date,Employer,City_State,Status,Job_Title,Wage_Offer,Country,test,test2,test3
525328,2018-01-09,Vi Department Of Education,"St. Thomas,",Certified,"Middle School Teachers, Except Special and Career/Technical Educ, Level I","45,300.00/Year",Jamaica,Vi Department Of Education,Vi Department Of Education,Education
631305,2018-01-09,"Axon Enterprise, Inc.","Seattle, WA",Certified,"Software Developers, Systems Software, Level IV","144,000.24/Year",Russia,Axon Enterprise Inc,Axon Enterprise,Inc
22892,2018-01-08,"David R. Olinger, D.c.","SAN MARCOS, CA",Certified,"CHIROPRACTORS, Level I","39,936.00/Year",Canada,David R Olinger Dc,David R Olinger,Dc
122460,2018-01-08,Teba Pharmacy Inc.,"Brooklyn, NY",Certified,"Pharmacy Aide, Level IV","29,910.00/Year",Egypt,Teba Pharmacy Inc,Teba Pharmacy Inc,Inc
146542,2018-01-08,Cybertec Inc,"Reston, VA",Certified,"Systems Analyst, Level II","90,646.00/Year",India,Cybertec Inc,Cybertec Inc,Inc
717383,2017-11-17,"Cts Technical Services, Inc.","Everett, WA",Certified,"Aerospace Engineers, Level IV","107,120.00/Year",United Kingdom,Cts Technical Services Inc,Cts Technical Services,Inc


In [45]:
test <- perm_data[grepl(",", perm_data[["Employer"]]),]
head(test)

,Date,Employer,City_State,Status,Job_Title,Wage_Offer,Country,test,test2,test3
631305,2018-01-09,"Axon Enterprise, Inc.","Seattle, WA",Certified,"Software Developers, Systems Software, Level IV","144,000.24/Year",Russia,Axon Enterprise Inc,Axon Enterprise,Inc.
22892,2018-01-08,"David R. Olinger, D.c.","SAN MARCOS, CA",Certified,"CHIROPRACTORS, Level I","39,936.00/Year",Canada,David R Olinger Dc,David R Olinger,D.c.
717383,2017-11-17,"Cts Technical Services, Inc.","Everett, WA",Certified,"Aerospace Engineers, Level IV","107,120.00/Year",United Kingdom,Cts Technical Services Inc,Cts Technical Services,Inc.
8182,2017-09-29,"El Charrito Of Homestead, Inc.","Homestead, FL",Certified,"Marketing Managers, Level I","60,778.00/Year",Bangladesh,El Charrito Of Homestead Inc,El Charrito Of Homestead,Inc.
10741,2017-09-29,"Epam Systems, Inc","Bellevue, WA",Certified,"Software Developers, Applications, Level I","98,000.00/Year",Belarus,Epam Systems Inc,Epam Systems,Inc
11568,2017-09-29,"Connectwise, Inc.","Tampa, FL",Certified,"Computer Occupations, All Other, Level III","81,806.00/Year",Belgium,Connectwise Inc,Connectwise,Inc.


In [84]:
test <- perm_data[grepl(",", perm_data[["Employer"]]),]
# lower case
test$test <- sapply(test$Employer, tolower)
# word after ","
test$test <- sapply(test$test, function(x) tail(strsplit(x, ",")[[1]], 1))
# special characters
test$test <- gsub("[[:punct:]]", "", test$test)

In [85]:
last_word_counts <- count(test, 'test')
last_word_counts <- last_word_counts[order(last_word_counts$freq,decreasing = TRUE),]
head(last_word_counts, 100)

# first will be remove dba (dba, D/b/a , d/b/a)
# second will be remove , + the following top 10 key words

,test,freq
920,inc,273627
3175,llc,55792
3640,llp,6161
4227,inc,4231
3860,pc,2874
3651,lp,2855
3681,ltd,2073
3791,na,1293
3838,pa,996
4390,llc,979


In [90]:
# test <- perm_data[grepl(",", perm_data[["Employer"]]),]
# # lower case
# test$test <- sapply(test$Employer, tolower)
# # word after ","
# test$test <- sapply(test$test, function(x) tail(strsplit(x, ",")[[1]], 1))
# # special characters
# test$test <- gsub("[[:punct:]]", "", test$test)
                    
test2 <- test[grepl("inc dba claxton poultry farms", test[["test"]]),]
head(test2)

,Date,Employer,City_State,Status,Job_Title,Wage_Offer,Country,test,test2,test3
57318,2017-07-06,"Norman W. Fries, Inc. D/b/a Claxton Poultry Farms","CLAXTON, GA",Denied,"Meat, Poultry and Fish Cutter and Trimmers, Level I","17,014.00/Year",China,inc dba claxton poultry farms,Norman W Fries,Inc. D/b/a Claxton Poultry Farms
57319,2017-07-06,"Norman W. Fries, Inc. D/b/a Claxton Poultry Farms","CLAXTON, GA",Denied,"Meat, Poultry and Fish Cutter and Trimmers, Level I","17,014.00/Year",China,inc dba claxton poultry farms,Norman W Fries,Inc. D/b/a Claxton Poultry Farms
57320,2017-07-06,"Norman W. Fries, Inc. D/b/a Claxton Poultry Farms","CLAXTON, GA",Denied,"Meat, Poultry and Fish Cutter and Trimmers, Level I","17,014.00/Year",China,inc dba claxton poultry farms,Norman W Fries,Inc. D/b/a Claxton Poultry Farms
57448,2017-06-28,"Norman W. Fries, Inc. D/b/a Claxton Poultry Farms","CLAXTON, GA",Denied,"Meat, Poultry and Fish Cutter and Trimmers, Level I","17,014.00/Year",China,inc dba claxton poultry farms,Norman W Fries,Inc. D/b/a Claxton Poultry Farms
57560,2017-06-26,"Norman W. Fries, Inc. D/b/a Claxton Poultry Farms","CLAXTON, GA",Certified,"HELPERS - PRODUCTION WORKERS, Level I","17,347.00/Year",China,inc dba claxton poultry farms,Norman W Fries,Inc. D/b/a Claxton Poultry Farms
57563,2017-06-26,"Norman W. Fries, Inc. D/b/a Claxton Poultry Farms","CLAXTON, GA",Certified,"HELPERS - PRODUCTION WORKERS, Level I","17,347.00/Year",China,inc dba claxton poultry farms,Norman W Fries,Inc. D/b/a Claxton Poultry Farms


In [65]:
a = c('axon enterprise, inc.', 'david r. olinger, d.c.', 'cts technical services, inc.')
s = tail(strsplit(a, ",")[[1]], 1)
s

[1] " inc."

In [119]:
# test <- perm_data[grepl("\bdba, D/b/a , d/b/a", perm_data[["Employer"]]),]
test <- perm_data
test$test <- sapply(test$Employer, tolower)
# test$test <- gsub("[[:punct:]]", "", test$test)
head(test)

,Date,Employer,City_State,Status,Job_Title,Wage_Offer,Country,test,test2,test3
525328,2018-01-09,Vi Department Of Education,"St. Thomas,",Certified,"Middle School Teachers, Except Special and Career/Technical Educ, Level I","45,300.00/Year",Jamaica,vi department of education,Vi Department Of Education,Vi Department Of Education
631305,2018-01-09,"Axon Enterprise, Inc.","Seattle, WA",Certified,"Software Developers, Systems Software, Level IV","144,000.24/Year",Russia,"axon enterprise, inc.",Axon Enterprise,Inc.
22892,2018-01-08,"David R. Olinger, D.c.","SAN MARCOS, CA",Certified,"CHIROPRACTORS, Level I","39,936.00/Year",Canada,"david r. olinger, d.c.",David R Olinger,D.c.
122460,2018-01-08,Teba Pharmacy Inc.,"Brooklyn, NY",Certified,"Pharmacy Aide, Level IV","29,910.00/Year",Egypt,teba pharmacy inc.,Teba Pharmacy Inc,Teba Pharmacy Inc.
146542,2018-01-08,Cybertec Inc,"Reston, VA",Certified,"Systems Analyst, Level II","90,646.00/Year",India,cybertec inc,Cybertec Inc,Cybertec Inc
717383,2017-11-17,"Cts Technical Services, Inc.","Everett, WA",Certified,"Aerospace Engineers, Level IV","107,120.00/Year",United Kingdom,"cts technical services, inc.",Cts Technical Services,Inc.


In [160]:
# " dba ", "(dba)", " d/b/a ", ",dba", "dba,", "/dba/", " dba/", "/dba", "db/a", "d/ba", "d-b-a", ";dba", "dba;", "dba:", "dba-", "-dba" ("d/b/a==d.b.a")
# norman w. fries, inc. d/b/a claxton poultry farms
test2 <- test[grepl("\\b[[:punct:]]*d[[:punct:]]*b[[:punct:]]*a[[:punct:]]*\\b", test[["test"]]),]

last_word_counts <- count(test2, 'test')
last_word_counts <- last_word_counts[order(last_word_counts$freq,decreasing = TRUE),]
head(last_word_counts, 100)

,test,freq
4783,"norman w. fries, inc. d/b/a claxton poultry farms",788
799,baha industries corp. dba open systems technologies,283
3264,jcg foods of georgia llc d/b/a koch foods,248
5739,"saber software, inc., d/b/a saber government solut",202
4972,pacesetter inc. dba st. jude medical crmd,122
3037,indu llc d/b/a intigrow,101
1581,cw professional services dba lochbridge,100
1218,cellco partnership d/b/a verizon wireless,94
3909,"lex nimble solutions inc dba ilbsg,inc",73
7439,"zylog systems limited dba zsl, inc.",73


# notes

In [18]:
sapply(lapply(original_data,unique), length)

id       Date   Employer City_State     Status  Job_Title Wage_Offer 
    734587       2957     154202      16644          5      61051     101446 
   Country 
       177